In [1]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));

import warnings
warnings.filterwarnings('ignore')

In [2]:
# code for linear regression models goes here
import statsmodels.api as sm
import pandas as pd

airport = pd.read_csv("data/Airfares.csv")

# define predictor variables (same as decision variables for LP)
predictors = ['COUPON', 'HI', 'DISTANCE']
X = airport[predictors] 

# define response variables for each model
Y_obj = airport['FARE']
Y_c1 = airport['PAX']
Y_c2 = airport['S_INCOME']
Y_c3 = airport['E_INCOME']

# add a constant to each model
# X = sm.add_constant(X) 

# fit the objective function and pull out coefficients
model_obj = sm.OLS(Y_obj, X).fit()
coefs_obj = model_obj.params

print_model_obj = model_obj.summary()
print(print_model_obj)
print(coefs_obj)


# fit the constraint 1
model_c1 = sm.OLS(Y_c1, X).fit()
coefs_c1 = model_c1.params

print_model_c1 = model_c1.summary()
print(print_model_c1)
print(coefs_c1)

# fit the constraint 2
model_c2 = sm.OLS(Y_c2, X).fit() 
coefs_c2 = model_c2.params

print_model_c2 = model_c2.summary()
print(print_model_c2)
print(coefs_c2)

# fit the constraint 3
model_c3 = sm.OLS(Y_c3, X).fit()
coefs_c3 = model_c3.params

print_model_c3 = model_c3.summary()
print(print_model_c3)
print(coefs_c3)

                                 OLS Regression Results                                
Dep. Variable:                   FARE   R-squared (uncentered):                   0.911
Model:                            OLS   Adj. R-squared (uncentered):              0.911
Method:                 Least Squares   F-statistic:                              2165.
Date:                Sat, 05 Oct 2019   Prob (F-statistic):                        0.00
Time:                        07:33:49   Log-Likelihood:                         -3439.5
No. Observations:                 638   AIC:                                      6885.
Df Residuals:                     635   BIC:                                      6898.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [4]:
# code for Pyomo and nicely formatted output goes here

# Unfold to see the Pyomo solution with arrays of decision variables
from pyomo.environ import *
import pandas as pd
import numpy as np

# setup dat
#predictors = ['coupon', 'HI', 'distance']

obj_coefs = dict(zip(predictors, coefs_obj))

con1_coefs_pax = dict(zip(predictors, coefs_c1))
con2_coefs_s_income = dict(zip(predictors, coefs_c2))
con3_coefs_e_income = dict(zip(predictors, coefs_c3))

# Concrete Model
M = ConcreteModel(name="Airfare")

# Decision Variables
M.x = Var(predictors, domain=NonNegativeReals)

# Objective
M.fare = Objective(expr=sum(obj_coefs[p] * M.x[p] for p in predictors), sense=maximize)

M.con1_pax = Constraint(expr=sum(con1_coefs_pax[p] * M.x[p] for p in predictors) <= 20000)
M.con2_s_income = Constraint(expr=sum(con2_coefs_s_income[p] * M.x[p] for p in predictors) <= 30000)
M.con3_e_income = Constraint(expr=sum(con3_coefs_e_income[p] * M.x[p] for p in predictors) >= 30000)
M.con4_coupon = Constraint(expr=M.x['COUPON'] <= 1.5)
M.con5_HI = Constraint(expr=M.x['HI'] >= 4000)
M.con6_HI = Constraint(expr=M.x['HI'] <= 8000)
M.con7_distance = Constraint(expr=M.x['DISTANCE'] >= 500)
M.con8_distance = Constraint(expr=M.x['DISTANCE'] <= 1000)

# Solve
solver = SolverFactory('glpk')
solver.solve(M)

# display solution
import babel.numbers as numbers  # needed to display as currency
print("Max airfare = ", numbers.format_currency(M.fare(), 'USD',
                                               locale='en_US'))

print("\nCoupon: {:0.2f}".format(M.x['COUPON']()))
print("\nHI: {:0.2f}".format(M.x['HI']()))
print("\nDistance: {:0.2f}".format(M.x['DISTANCE']()))

Max airfare =  $203.55

Coupon: 1.14

HI: 8000.00

Distance: 1000.00


In [5]:
# code to generate and display sensitivity report goes here
# write the model to a sensitivity report
M.write('model2.lp', io_options={'symbolic_solver_labels': True})
!glpsol -m model2.lp --lp --ranges sensit2.sen

# widen browser and/or close TOC to see sensitivity report
# or open sensit2.sen in a text editor
import numpy as np
f = open('sensit2.sen', 'r')
file_contents = f.read()
print(file_contents)
f.close()

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 -m model2.lp --lp --ranges sensit2.sen
Reading problem data from 'model2.lp'...
9 rows, 4 columns, 15 non-zeros
56 lines were read
GLPK Simplex Optimizer, v4.65
9 rows, 4 columns, 15 non-zeros
Preprocessing...
2 rows, 3 columns, 6 non-zeros
Scaling...
 A: min|aij| =  1.020e+00  max|aij| =  2.091e+04  ratio =  2.050e+04
GM: min|aij| =  7.309e-01  max|aij| =  1.368e+00  ratio =  1.872e+00
EQ: min|aij| =  5.342e-01  max|aij| =  1.000e+00  ratio =  1.872e+00
Constructing initial basis...
Size of triangular part is 2
      0: obj =   8.885866366e+01 inf =   2.215e+04 (1)
      3: obj =   1.739717779e+02 inf =   0.000e+00 (0)
*     4: obj =   2.035540468e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40412 bytes)
Write sensitivity analysis report to 'sensit2.sen'...
GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                               